In [1]:
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model, to_categorical

import numpy as np
import os
import math

In [3]:
batch_size=128
epochs=200
data_augmentation=True
num_classes=10

subtract_pixel_mean=True

n=3
version=1

if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

In [4]:
model_type = 'ResNet%dv%d' % (depth, version)

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 227s 1us/step


In [6]:
# input image dimensions.
input_shape = x_train.shape[1:]

# normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [7]:
# if subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [8]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [9]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-2
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [10]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):

    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [11]:
def resnet_v1(input_shape, depth, num_classes=10):

    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, in [a])')
    # start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            # first layer but not first stack
            if stack > 0 and res_block == 0:  
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            # first layer but not first stack
            if stack > 0 and res_block == 0:
                # linear projection residual shortcut
                # connection to match changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
def resnet_v2(input_shape, depth, num_classes=10):
    
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 110 in [b])')
    # start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU
    # on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                # first layer and first stage
                if res_block == 0:  
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                # first layer but not first stage
                if res_block == 0:
                    # downsample
                    strides = 2 

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection
                # to match changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = add([x, y])

        num_filters_in = num_filters_out

    # add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [13]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['acc'])
model.summary()

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch

                                                                                                  
 batch_normalization_9 (BatchNo  (None, 16, 16, 32)  128         ['conv2d_10[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 activation_9 (Activation)      (None, 16, 16, 32)   0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_11 (Conv2D)             (None, 16, 16, 32)   9248        ['activation_9[0][0]']           
                                                                                                  
 batch_normalization_10 (BatchN  (None, 16, 16, 32)  128         ['conv2d_11[0][0]']              
 ormalization)                                                                                    
          

                                                                                                  
 activation_18 (Activation)     (None, 8, 8, 64)     0           ['add_8[0][0]']                  
                                                                                                  
 average_pooling2d (AveragePool  (None, 1, 1, 64)    0           ['activation_18[0][0]']          
 ing2D)                                                                                           
                                                                                                  
 flatten (Flatten)              (None, 64)           0           ['average_pooling2d[0][0]']      
                                                                                                  
 dense (Dense)                  (None, 10)           650         ['flatten[0][0]']                
                                                                                                  
Total para

C:\Users\sarim\anaconda3\envs\work\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
print(model_type)

ResNet20v1


In [15]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [16]:
# prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [17]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # this will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    steps_per_epoch =  math.ceil(len(x_train) / batch_size)
    # fit the model on the batches generated by datagen.flow().
    model.fit(x=datagen.flow(x_train, y_train, batch_size=batch_size),
              verbose=1,
              epochs=epochs,
              validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch,
              callbacks=callbacks)

Using real-time data augmentation.
Learning rate:  0.001
Epoch 1/200
391/391 [==============================] - ETA: 0s - loss: 1.6377 - acc: 0.4630
Epoch 1: val_acc improved from -inf to 0.47250, saving model to D:\Data science\Machine Learning\Deep learning\Tensorflow - new\saved_models\cifar10_ResNet20v1_model.001.h5
391/391 [==============================] - 30s 62ms/step - loss: 1.6377 - acc: 0.4630 - val_loss: 1.6323 - val_acc: 0.4725 - lr: 0.0010
Learning rate:  0.001
Epoch 2/200
391/391 [==============================] - ETA: 0s - loss: 1.2642 - acc: 0.5979
Epoch 2: val_acc improved from 0.47250 to 0.59860, saving model to D:\Data science\Machine Learning\Deep learning\Tensorflow - new\saved_models\cifar10_ResNet20v1_model.002.h5
391/391 [==============================] - 23s 59ms/step - loss: 1.2642 - acc: 0.5979 - val_loss: 1.2790 - val_acc: 0.5986 - lr: 0.0010
Learning rate:  0.001
Epoch 3/200
390/391 [============================>.] - ETA: 0s - loss: 1.0827 - acc: 0.6680
Ep

391/391 [==============================] - 46s 118ms/step - loss: 0.4843 - acc: 0.8834 - val_loss: 0.8623 - val_acc: 0.7818 - lr: 0.0010
Learning rate:  0.001
Epoch 25/200
391/391 [==============================] - ETA: 0s - loss: 0.4833 - acc: 0.8845
Epoch 25: val_acc improved from 0.80650 to 0.81040, saving model to D:\Data science\Machine Learning\Deep learning\Tensorflow - new\saved_models\cifar10_ResNet20v1_model.025.h5
391/391 [==============================] - 47s 120ms/step - loss: 0.4833 - acc: 0.8845 - val_loss: 0.7310 - val_acc: 0.8104 - lr: 3.1623e-04
Learning rate:  0.001
Epoch 26/200
391/391 [==============================] - ETA: 0s - loss: 0.4767 - acc: 0.8854
Epoch 26: val_acc did not improve from 0.81040
391/391 [==============================] - 46s 118ms/step - loss: 0.4767 - acc: 0.8854 - val_loss: 0.9134 - val_acc: 0.7640 - lr: 0.0010
Learning rate:  0.001
Epoch 27/200
391/391 [==============================] - ETA: 0s - loss: 0.4743 - acc: 0.8878
Epoch 27: val_ac

Learning rate:  0.001
Epoch 50/200
391/391 [==============================] - ETA: 0s - loss: 0.3918 - acc: 0.9181
Epoch 50: val_acc did not improve from 0.86830
391/391 [==============================] - 40s 102ms/step - loss: 0.3918 - acc: 0.9181 - val_loss: 0.8358 - val_acc: 0.8128 - lr: 3.1623e-04
Learning rate:  0.001
Epoch 51/200
391/391 [==============================] - ETA: 0s - loss: 0.3874 - acc: 0.9194
Epoch 51: val_acc did not improve from 0.86830
391/391 [==============================] - 40s 103ms/step - loss: 0.3874 - acc: 0.9194 - val_loss: 0.6211 - val_acc: 0.8537 - lr: 0.0010
Learning rate:  0.001
Epoch 52/200
391/391 [==============================] - ETA: 0s - loss: 0.3867 - acc: 0.9186
Epoch 52: val_acc did not improve from 0.86830
391/391 [==============================] - 41s 104ms/step - loss: 0.3867 - acc: 0.9186 - val_loss: 0.6488 - val_acc: 0.8473 - lr: 0.0010
Learning rate:  0.001
Epoch 53/200
391/391 [==============================] - ETA: 0s - loss: 0.387

Learning rate:  0.001
Epoch 77/200
391/391 [==============================] - ETA: 0s - loss: 0.3583 - acc: 0.9308
Epoch 77: val_acc did not improve from 0.87180
391/391 [==============================] - 44s 113ms/step - loss: 0.3583 - acc: 0.9308 - val_loss: 0.7885 - val_acc: 0.8248 - lr: 0.0010
Learning rate:  0.001
Epoch 78/200
391/391 [==============================] - ETA: 0s - loss: 0.3556 - acc: 0.9317
Epoch 78: val_acc did not improve from 0.87180
391/391 [==============================] - 46s 116ms/step - loss: 0.3556 - acc: 0.9317 - val_loss: 0.6288 - val_acc: 0.8626 - lr: 0.0010
Learning rate:  0.001
Epoch 79/200
391/391 [==============================] - ETA: 0s - loss: 0.3545 - acc: 0.9324
Epoch 79: val_acc did not improve from 0.87180
391/391 [==============================] - 46s 116ms/step - loss: 0.3545 - acc: 0.9324 - val_loss: 0.7661 - val_acc: 0.8333 - lr: 3.1623e-04
Learning rate:  0.001
Epoch 80/200
391/391 [==============================] - ETA: 0s - loss: 0.358

391/391 [==============================] - ETA: 0s - loss: 0.1961 - acc: 0.9822
Epoch 102: val_acc did not improve from 0.90530
391/391 [==============================] - 44s 113ms/step - loss: 0.1961 - acc: 0.9822 - val_loss: 0.5248 - val_acc: 0.8996 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 103/200
391/391 [==============================] - ETA: 0s - loss: 0.1966 - acc: 0.9812
Epoch 103: val_acc did not improve from 0.90530
391/391 [==============================] - 44s 113ms/step - loss: 0.1966 - acc: 0.9812 - val_loss: 0.5027 - val_acc: 0.9014 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 104/200
391/391 [==============================] - ETA: 0s - loss: 0.1939 - acc: 0.9815
Epoch 104: val_acc did not improve from 0.90530
391/391 [==============================] - 45s 116ms/step - loss: 0.1939 - acc: 0.9815 - val_loss: 0.4911 - val_acc: 0.9044 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 105/200
391/391 [==============================] - ETA: 0s - loss: 0.1915 - acc: 0.9826
Ep

Learning rate:  1e-05
Epoch 128/200
391/391 [==============================] - ETA: 0s - loss: 0.1594 - acc: 0.9899
Epoch 128: val_acc did not improve from 0.90590
391/391 [==============================] - 20s 51ms/step - loss: 0.1594 - acc: 0.9899 - val_loss: 0.5094 - val_acc: 0.9059 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 129/200
390/391 [============================>.] - ETA: 0s - loss: 0.1586 - acc: 0.9897
Epoch 129: val_acc did not improve from 0.90590
391/391 [==============================] - 20s 51ms/step - loss: 0.1586 - acc: 0.9898 - val_loss: 0.5082 - val_acc: 0.9044 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 130/200
391/391 [==============================] - ETA: 0s - loss: 0.1586 - acc: 0.9900
Epoch 130: val_acc did not improve from 0.90590
391/391 [==============================] - 20s 51ms/step - loss: 0.1586 - acc: 0.9900 - val_loss: 0.5112 - val_acc: 0.9055 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 131/200
391/391 [==============================] - ETA: 0s -

391/391 [==============================] - 20s 51ms/step - loss: 0.1525 - acc: 0.9917 - val_loss: 0.5175 - val_acc: 0.9038 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 155/200
390/391 [============================>.] - ETA: 0s - loss: 0.1525 - acc: 0.9913
Epoch 155: val_acc did not improve from 0.90630
391/391 [==============================] - 20s 51ms/step - loss: 0.1525 - acc: 0.9913 - val_loss: 0.5155 - val_acc: 0.9044 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 156/200
391/391 [==============================] - ETA: 0s - loss: 0.1517 - acc: 0.9921
Epoch 156: val_acc did not improve from 0.90630
391/391 [==============================] - 20s 51ms/step - loss: 0.1517 - acc: 0.9921 - val_loss: 0.5156 - val_acc: 0.9030 - lr: 3.1623e-06
Learning rate:  1e-05
Epoch 157/200
391/391 [==============================] - ETA: 0s - loss: 0.1530 - acc: 0.9908
Epoch 157: val_acc did not improve from 0.90630
391/391 [==============================] - 20s 51ms/step - loss: 0.1530 - acc: 0.9908 - 

391/391 [==============================] - 20s 51ms/step - loss: 0.1484 - acc: 0.9918 - val_loss: 0.5174 - val_acc: 0.9037 - lr: 3.1623e-06
Learning rate:  5e-07
Epoch 182/200
391/391 [==============================] - ETA: 0s - loss: 0.1485 - acc: 0.9919
Epoch 182: val_acc did not improve from 0.90630
391/391 [==============================] - 20s 51ms/step - loss: 0.1485 - acc: 0.9919 - val_loss: 0.5180 - val_acc: 0.9031 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 183/200
390/391 [============================>.] - ETA: 0s - loss: 0.1480 - acc: 0.9919
Epoch 183: val_acc did not improve from 0.90630
391/391 [==============================] - 20s 52ms/step - loss: 0.1480 - acc: 0.9919 - val_loss: 0.5166 - val_acc: 0.9035 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 184/200
390/391 [============================>.] - ETA: 0s - loss: 0.1484 - acc: 0.9919
Epoch 184: val_acc did not improve from 0.90630
391/391 [==============================] - 20s 51ms/step - loss: 0.1484 - acc: 0.9919 - 

In [18]:
# score trained model
scores = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.5176630616188049
Test accuracy: 0.9025999903678894
